# Second cleaning Script 

* Contract flipped with department
* Salaries into numerical

In [193]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [194]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999


import time
import unicodedata

import datetime
from datetime import timedelta, datetime

import csv
import os

import sys
sys.path.append('./lib/')
from functions import *
from tqdm import tnrange, tqdm_notebook
from unidecode import unidecode

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import glob

%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style("darkgrid")

import locale
locale.setlocale(locale.LC_ALL, 'es_ES')

'es_ES'

# Import data

In [195]:
period = '20180210-3'

In [196]:
df = pd.read_csv('../data/output/01_scraped_data_clean_first_{}.csv'.format(period), encoding='utf-8', low_memory=False)

In [197]:
df.shape

(1954998, 23)

# Null names?

In [200]:
df.loc[pd.isnull(df['givenn'])]

array([], dtype=object)

In [201]:
df.loc[df['givenn'] == '-1', 'url'].unique()

array([ 'http://www.gobiernotransparentechile.cl/directorio/entidad/2/5/per_planta/Ao-2007',
       'http://www.gobiernotransparentechile.cl/directorio/entidad/2/5/per_planta/Ao-2007?x=0&y=0&page_number=2&sort=id&direction=asc',
       'http://www.gobiernotransparentechile.cl/directorio/entidad/2/5/per_planta/Ao-2007?x=0&y=0&page_number=3&sort=id&direction=asc',
       'http://www.gobiernotransparentechile.cl/directorio/entidad/2/5/per_planta/Ao-2006',
       'http://www.gobiernotransparentechile.cl/directorio/entidad/2/5/per_planta/Ao-2006?x=0&y=0&page_number=2&sort=id&direction=asc',
       'http://www.gobiernotransparentechile.cl/directorio/entidad/2/5/per_planta/Ao-2006?x=0&y=0&page_number=3&sort=id&direction=asc',
       'http://www.gobiernotransparentechile.cl/directorio/entidad/2/5/per_contrata/Ao-2008',
       'http://www.gobiernotransparentechile.cl/directorio/entidad/2/5/per_contrata/Ao-2008?x=0&y=0&page_number=2&sort=id&direction=asc',
       'http://www.gobiernotransparente

# Shifted table because of extra column 

In [175]:
df['lastn'].value_counts()

gonzalez                                                                                                                                      32880
rojas                                                                                                                                         23393
muñoz                                                                                                                                         22176
diaz                                                                                                                                          18268
profesionales                                                                                                                                 16760
silva                                                                                                                                         14542
soto                                                                                                            

## Gendarmeria

In [176]:
scope = ((df['lastn'].str.contains('profesio')) | (df['lastn'].str.contains('administr'))) & \
                    (df['contract'].str.contains('Gendarme'))
df.loc[scope].sample(1)

,entity,department,contract,year,month,group,lastn,lastn2,givenn,degree,qualif,position,region,specials,currency,salary,overt,start_date,end_date,obs,url,other,filename
1179452,Ministerio de Justicia y Derechos Humanos,per_contrata,Gendarmería de Chile,Agosto,allyear,CONTRATA,profesionales,zuñiga,bulnes,LUIS ALBERTO,13,CONTADOR AUDITOR,ENCARGADO ADMINISTRATIVO UNIDAD,V,"2, 9",PESOS,994.163,805.441,,01/01/2011,31/12/2011,12,../data/output/scraped_data_20180210-3.csv


In [177]:
df.loc[scope, 'month'], df.loc[scope, 'year'] = df.loc[scope, 'year'], df.loc[scope, 'month']


col_shift = list(df.columns[5:-5])
for i,j in zip(col_shift, col_shift[1:]):
    df.loc[scope, i] = df.loc[scope, j].copy()

col_shift = list(['start_date', 'end_date', 'obs', 'url'])
for i,j in zip(col_shift, col_shift[2:]):
    df.loc[scope, i] = df.loc[scope, j].copy()
    #print(i,j)

df.loc[scope, ['obs', 'url']] = None

## Vialidad 

In [178]:
scope = ((df['lastn'].str.contains('profesio')) | (df['lastn'].str.contains('administr'))) & \
                    (df['contract'].str.contains('Vialidad')) & (df['group'] == '-1')

In [179]:
#df.loc[scope].sample(5)

In [180]:
col_shift = list(df.columns[6:10])
for i,j in zip(col_shift, col_shift[1:]):
    df.loc[scope, i] = df.loc[scope, j].copy()
    #print(i,j)

col_shift = list(df.columns[10:-1])
for i,j in zip(col_shift, col_shift[1:]):
    df.loc[scope, i] = df.loc[scope, j].copy()
    #print(i,j)

df.loc[scope, ['degree', 'other']] = None

## What's left

In [181]:
scope = ((df['lastn'].str.contains('profesio')) | df['lastn'].str.contains('administra'))
df.loc[scope, 'contract'].value_counts(dropna=False) / df.shape[0]

Servicio de Registro Civil e Identificación (SRCEI)    0.003428
Dirección de Vialidad                                  0.003257
Servicio de Impuestos Internos (SII)                   0.002721
Instituto de Previsión Social (IPS)                    0.002063
Consejo de Defensa del Estado                          0.000585
Dirección de Obras Portuarias (DOP)                    0.000378
Subsecretaría de Relaciones Exteriores                 0.000191
SERVIU V Región                                        0.000178
Servicio Nacional del Adulto Mayor (SENAMA)            0.000145
Dotación a contrata                                    0.000104
Instituto Nacional de la Juventud (INJUV)              0.000099
Dotación de planta                                     0.000088
Gobernación de Los Andes                               0.000061
SERVIU VII Región                                      0.000038
Name: contract, dtype: float64

# Contract filpped with Department 

In [182]:
df.loc[df['department'].str.contains('per_')].head(5)

,entity,department,contract,year,month,group,lastn,lastn2,givenn,degree,qualif,position,region,specials,currency,salary,overt,start_date,end_date,obs,url,other,filename
174046,Ministerio del Interior y Seguridad Pública,per_contrata,Gobernación de Los Andes,2017,allyear,1,profesional,aedo,torres,BORIS CRISTIAN,,ADMINISTRACION PUBLICA,PROFESIONAL,5,1,Pesos,975.647,NO,15/01/2017,31/12/2017,PAGO PROPORCIONAL A LOS DÍAS TRABAJADOS EN EL...,http://www.interior.gov.cl/transparenciaactiva...,../data/output/scraped_data_20180210-3.csv
174047,Ministerio del Interior y Seguridad Pública,per_contrata,Gobernación de Los Andes,2017,allyear,2,profesional,cubillos,cubillos,NICOLAS ANDRES,,INGENIERO EN REVENCION DE RIESGO Y MEDIO AMBI...,COORDINADOR DELEGADO DEL PASO FRONTERIZO LOS ...,5,1,Pesos,1.829.338,NO,01/01/2017,31/12/2017,,http://www.interior.gov.cl/transparenciaactiva...,../data/output/scraped_data_20180210-3.csv
174048,Ministerio del Interior y Seguridad Pública,per_contrata,Gobernación de Los Andes,2017,allyear,3,profesional,inostroza,arancibia,IVAN ENRIQUE,,ABOGADO,ASESOR JURIDICO DE LA GOBERNACIÓN PROVINCIAL ...,5,1,Pesos,1.829.338,NO,01/01/2017,31/12/2017,,http://www.interior.gov.cl/transparenciaactiva...,../data/output/scraped_data_20180210-3.csv
174049,Ministerio del Interior y Seguridad Pública,per_contrata,Gobernación de Los Andes,2017,allyear,4,profesional,rosende,bey,VERONICA CECILIA,,TRABAJADOR SOCIAL,PROFESIONAL,5,2,Pesos,1.849.118,NO,01/01/2017,31/12/2017,,http://www.interior.gov.cl/transparenciaactiva...,../data/output/scraped_data_20180210-3.csv
174050,Ministerio del Interior y Seguridad Pública,per_contrata,Gobernación de Los Andes,2017,allyear,5,profesional,hernandez,flores,MAURICIO BRYAM,,CONTADOR;INGENIERO INFORMATICO,ENCARGADO DE ADMINISTRACION Y FINANZAS,5,1,Pesos,1.680.841,NO,01/01/2017,31/12/2017,,http://www.interior.gov.cl/transparenciaactiva...,../data/output/scraped_data_20180210-3.csv


In [183]:
flipColumns(df, 'department', 'contract', 'per_')

140,375 rows of data with flipped department and contract (7.2%)
Involved department:
Servicio de Impuestos Internos (SII)                     47381
Dirección de Vialidad                                    35684
Gendarmería de Chile                                     30606
Servicio de Registro Civil e Identificación (SRCEI)       8988
Instituto de Previsión Social (IPS)                       6874
Parque Metropolitano de Santiago                          2564
Consejo de Defensa del Estado                             1920
Dirección de Obras Portuarias (DOP)                       1110
Superintendencia de Electricidad y Combustibles (SEC)     1001
SERVIU VII Región                                         1001
Subsecretaría de Economía                                  662
SERVIU V Región                                            470
Subsecretaría de Relaciones Exteriores                     463
Dirección General de Crédito Prendario (DICREP)            408
Servicio Nacional del Adulto May

In [185]:
df.loc[df['department'].str.contains('Dotaci')]

,entity,department,contract,year,month,group,lastn,lastn2,givenn,degree,qualif,position,region,specials,currency,salary,overt,start_date,end_date,obs,url,other,filename,auxdepartment,auxcontract


What's left could be dummy data (0.1% of rows)

# Salary into int 

## Cleaning script 

In [186]:
df['salary1'] = df['salary'].str.replace('.', '')

clean_list = ['.', ',', ' ', '\t', '$', '-', 'Pesos']
for c in clean_list:
    df['salary1'] = df['salary1'].str.replace(c, '')

clean_vals = ['', '0', '1', '10']
for c in clean_vals:
    df.loc[df['salary1'] == c] = None


## Checks 

In [187]:
df['salary1'].sort_values().head()

1740429     100000
724037     1000000
728733     1000000
723844     1000000
724231     1000000
Name: salary1, dtype: object

In [188]:
df['salary1'].sort_values().tail()

1954993    None
1954994    None
1954995    None
1954996    None
1954997    None
Name: salary1, dtype: object

## Results 

In [189]:
df['salary1'] = pd.to_numeric(df['salary1'], errors='coerce')

In [190]:
pd.isnull(df['salary1']).value_counts() / df.shape[0]

False    0.839712
True     0.160288
Name: salary1, dtype: float64

16.34% of salaries are null

In [191]:
df.loc[pd.isnull(df['salary1']) & (pd.notnull(df['entity'])), 'year'].value_counts()

Año 2008      14851
Año 2007      13828
Año 2006       7618
Año 2010       4309
Año 2009       3455
Año 2015       1800
Año 2017       1627
Año 2016       1525
Año 2014       1475
Año 2013       1400
Año 2012       1347
Año 2011       1253
Diciembre       784
Noviembre       778
Octubre         772
Septiembre      764
Enero           758
Agosto          751
Febrero         750
Julio           738
Junio           723
Mayo            714
Abril           690
Marzo           654
A�o 2007        200
A�o 2008        200
A�o 2009        200
A�o 2006        200
2016             17
2017             16
2015             16
2013             16
2014             15
2012             15
2011             14
2009             13
2010             12
Name: year, dtype: int64

# Print output 

In [192]:
df.to_csv('../data/output/scraped_data_clean_second_{}.csv'.format(period), encoding='utf-8', index=False)